In [3]:
import torch
print(torch.__version__)  # 查看 PyTorch 版本
print(torch.version.cuda)  # 查看 CUDA 版本
print(torch.cuda.is_available())  # 检查 CUDA 是否可用

2.5.1+cu124
12.4
True


In [1]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info


/opt/conda/envs/video-r1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-15 15:32:19,549] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/conda/envs/video-r1/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/conda/envs/video-r1/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [2]:
# model_path= "/data/guojian.li/Weight/Qwen2.5-VL-7B-COT-SFT/"
model_path = "/data/guojian.li/Weight/Video-R1/"

In [3]:
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_path, torch_dtype="auto", device_map="cuda:7"
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.24s/it]


In [4]:
# 检查模型的参数数据类型
for name, param in model.named_parameters():
    print(f"Parameter {name} dtype: {param.dtype}")

Parameter visual.patch_embed.proj.weight dtype: torch.float32
Parameter visual.blocks.0.norm1.weight dtype: torch.float32
Parameter visual.blocks.0.norm2.weight dtype: torch.float32
Parameter visual.blocks.0.attn.qkv.weight dtype: torch.float32
Parameter visual.blocks.0.attn.qkv.bias dtype: torch.float32
Parameter visual.blocks.0.attn.proj.weight dtype: torch.float32
Parameter visual.blocks.0.attn.proj.bias dtype: torch.float32
Parameter visual.blocks.0.mlp.gate_proj.weight dtype: torch.float32
Parameter visual.blocks.0.mlp.gate_proj.bias dtype: torch.float32
Parameter visual.blocks.0.mlp.up_proj.weight dtype: torch.float32
Parameter visual.blocks.0.mlp.up_proj.bias dtype: torch.float32
Parameter visual.blocks.0.mlp.down_proj.weight dtype: torch.float32
Parameter visual.blocks.0.mlp.down_proj.bias dtype: torch.float32
Parameter visual.blocks.1.norm1.weight dtype: torch.float32
Parameter visual.blocks.1.norm2.weight dtype: torch.float32
Parameter visual.blocks.1.attn.qkv.weight dtype: t

In [4]:
processor = AutoProcessor.from_pretrained(model_path)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


## 图片

In [5]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "/data/guojian.li/Project/Video-R1/my_demo/data/image.png",
            },
            {"type": "text", "text": "Are there the same number of cylinders and gray matte balls?"},
        ],
    }
]

In [ ]:
# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=256)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)
# cot

["To determine if there are the same number of cylinders and gray matte balls in the image, let's count each type separately.\n\nFirst, let's count the cylinders:\n- There is one small cyan cylinder.\n- There is one small green cylinder.\n\nSo, there are 2 cylinders in total.\n\nNext, let's count the gray matte balls:\n- There is one small gray matte ball.\n\nSo, there is 1 gray matte ball in total.\n\nComparing the counts:\n- Cylinders: 2\n- Gray matte balls: 1\n\nSince 2 is not equal to 1, there are not the same number of"]


In [7]:
# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=256)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)
# r1

["To determine if there are the same number of cylinders and gray matte balls in the image, let's count each type separately.\n\nFirst, let's count the cylinders:\n- There is a small cyan cylinder on the left side of the image.\n- There is a green cylinder near the center-right of the image.\nSo, there are 2 cylinders in total.\n\nNext, let's count the gray matte balls:\n- There is one small gray matte ball located towards the back of the image.\nSo, there is 1 gray matte ball in total.\n\nNow, we compare the counts:\n- The number of cylinders is 2.\n- The number of gray matte balls is 1.\n\nSince 2 is not equal to 1, there are not the same number of cylinders and gray matte balls.\n\nTherefore, the answer is no."]


## 视频

In [14]:
# Messages containing a video url and a text query
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "/data/guojian.li/Project/Video-R1/my_demo/data/emotion2.mp4",
                "fps": 1.0,
            },
            {"type": "text", "text": "Describe this video."},
        ],
    }
]

In [15]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs, video_kwargs = process_vision_info(messages, return_video_kwargs=True)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    
    padding=True,
    return_tensors="pt",
    **video_kwargs,
)
inputs = inputs.to(model.device)
inputs

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198, 151652, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         15165

In [10]:
# Inference
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['The video appears to be a scene from a movie or TV show, featuring two characters in what looks like an indoor setting with warm lighting and wooden beams on the ceiling. The character in the foreground is wearing a brown jacket over a pink shirt and seems to be speaking to someone off-camera. The subtitles at the bottom of the screen indicate that one character is asking another named James to lower his voice because he\'s "freaking out Jay." This suggests a tense or serious conversation taking place, possibly involving a third person named Jay who might be feeling distressed by the volume of the speaker. The overall atmosphere of the scene feels dramatic and focused']


In [16]:
# Inference
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

["The video appears to be a scene from a movie or TV show, featuring two characters in what looks like an indoor setting with warm lighting and wooden beams on the ceiling. One character is wearing a brown jacket over a red shirt, while the other character's back is turned to the camera. The subtitles suggest that one character is asking the other to lower their voice because they are scaring someone named Jay. The atmosphere seems tense, possibly indicating a moment of conflict or urgency between the characters. The background includes some artwork on the wall, adding to the indoor setting's ambiance."]


In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-7B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processor
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

: 